In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [2]:
base_url = "https://www.magicbricks.com/property-for-rent-in-jaipur-pppfr/page-{}"

In [3]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"}

In [1]:
rents= []
locations= []
bhk=[]
furnishing=[]
tenant_preferred=[]
carpet_area=[]
floors=[]

In [5]:
def clean_floor(floor_value):
    if "Ground" in floor_value:
        return 0
    if "Lower Basement" in floor_value:
        return -1
    if "Upper Basement" in floor_value:
        return -1
    match = re.search(r"(\d+) out of \d+", floor_value)
    return match.group(1) if match else "N/A"

In [6]:
max_pages = 50

In [7]:
for page in range(1, max_pages + 1):
    url = base_url.format(page)
    

    response = requests.get(url, headers=headers)

    
    if response.status_code != 200:
        print(f"Failed to load page {page}. Stopping...")
        break

    
    soup = BeautifulSoup(response.text, "html.parser")

    
    listings = soup.find_all("div", class_="mb-srp__list")

    for listing in listings:
        
        rent_element = listing.find("div", class_="mb-srp__card__price--amount")
        rent = rent_element.text.strip() if rent_element else "N/A"


        details = listing.find_all("div", class_="mb-srp__card__summary__list--item")

        
        bed, furnish, tenant, area, floor, location =  "N/A", "N/A", "N/A", "N/A", "N/A", "N/A"

        for detail in details:
            text = detail.text.strip()
            if "Bathroom" in text:
                bed = text.replace("Bathroom", "").strip()
            elif "Furnishing" in text:
                furnish = text.replace("Furnishing", "").strip()
            elif "Tenant Preferred" in text:
                tenant = text.replace("Tenant Preferred", "").strip()
            elif "Carpet Area" in text:
                area = text.replace("Carpet Area", "").strip()
            elif "Super Area" in text:
                area = text.replace("Super Area", "").strip()
            elif "Floor" in text:
                floor = clean_floor(text.replace("Floor", "").strip())

        
        title_element = listing.find("h2", class_="mb-srp__card--title")
        if title_element:
            title_text = title_element.text.strip()
            if "Rent in" in title_text:
                location = title_text.split("Rent in")[-1].strip()

        
        rents.append(rent)
        bhk.append(bed)
        furnishing.append(furnish)
        tenant_preferred.append(tenant)
        carpet_area.append(area)
        locations.append(location)
        floors.append(floor)

    
    time.sleep(2)


In [8]:
df = pd.DataFrame({
    "Rent": rents,
    "BHK": bhk,
    "Furnishing": furnishing,
    "Tenant Preferred": tenant_preferred,
    "Carpet Area": carpet_area,
    "Location": locations,
    "Floor": floors
})

In [9]:
df.to_excel("jaipur_rent.xlsx", index=False)

In [10]:
print("Excel file saved successfully!")

Excel file saved successfully!
